In [1]:
from features_labels_extract import extract_features_and_labels

In [12]:
import yfinance as yf
# Load Apple Stock Data (Ensure Close & Volume columns)
data = yf.download("TGT", start="2000-01-01", end="2024-01-01")[["Close", "Volume"]].dropna()

# Extract features and labels for Momentum strategy
X, y = extract_features_and_labels("TGT", data, strategy="momentum")

# Check Output Shapes
print("Feature Shape:", X.shape)  # Expected: (num_samples, num_features, num_days)
print("Label Shape:", y.shape)  # Expected: (num_samples,)


[*********************100%***********************]  1 of 1 completed


Final dataset size: 197 samples
Feature Shape: (197, 3, 30)
Label Shape: (197,)
